In [84]:
import numpy as np
from collections import Counter

def data_loader():
    # load data
    with open('train','r') as file:
        train_data = file.read().split('\n')[:-1]
    with open('test','r') as file:
        test_data = file.read().split('\n')[:-1]
    return train_data, test_data

def parser(datum):
    # extract labels and words
    email_addr, label, words = datum.split(' ',2)
    words = words.split()
    # transform words into dictionary
    word_dict = dict(zip([words[i] for i in range(0, len(words), 2)], [int(words[i+1]) for i in range(0, len(words), 2)]))
    # transform label into 0, 1
    if label == 'ham':
        label = 0
    elif label == 'spam':
        label = 1
    else: 
        raise ValueError
    return label, word_dict

def data_preprocessing(train_data, test_data):
    y_train = np.zeros(len(train_data))
    y_test = np.zeros(len(test_data))
    x_train = []
    x_test = []
    for i, datum in enumerate(train_data):
        label, word_dict = parser(datum)
        y_train[i] = label
        x_train.append(word_dict)
    for i, datum in enumerate(test_data):
        label, word_dict = parser(datum)
        y_test[i] = label
        x_test.append(word_dict)
    return x_train, y_train, x_test, y_test



In [90]:
def compute_empirical_pmf_y(y_train):
    # compte distribution P(y=1), P(y=0)
    p_y1=sum(y_train)/len(y_train) #spam
    p_y0=1-p_y1  #ham
    return p_y1, p_y0

def m_estimation_conditional_probability(x_train_frt, y_train, num_vocab, m):
    # compute P(x_j|y=1) and P(x_j|y=0) for j = 1, ..., d
    spam_row_index=np.where(y_train == 1)[0]
    ham_row_index=np.where(y_train == 0)[0]
    
    p_on_spam_m=[]
    p_on_ham_m=[]
    
    #num_vocab is d
    for i in np.arange(0,num_vocab):
        spam_train_frt=x_train_frt[spam_row_index,i]
        ham_train_frt=x_train_frt[ham_row_index,i]

        p_xi1_y1=(sum(spam_train_frt!=0)+m)/(len(spam_train_frt)+m*num_vocab) #p(xi=1|y=1)
        p_on_spam_m.append(p_xi1_y1)
        
        p_xi1_y0=(sum(ham_train_frt!=0)+m)/(len(ham_train_frt)+m*num_vocab) #p(xi=1|y=0)
        p_on_ham_m.append(p_xi1_y0)    

    return np.array(p_on_spam_m), np.array(p_on_ham_m)

def log_estimated_probability(p_spam, p_ham, p_on_spam_m, p_on_ham_m, x_frts):
    # compute log(P(y=1, x_1, x_2,..., x_n)) and log(P(y=0, x_1, x_2,..., x_n))
    #log(P(y, x_1, x_2,..., x_n)) = log(P(y=1)) + log(P(x_1 | y)) + ... + log(P(x_d | y))

    log_p_spam=[]
    log_p_ham=[]
    for i in np.arange(0,x_frts.shape[0]):    
        spam_index=np.where(x_test_frt[i]!=0)[0]
        ham_index=np.where(x_test_frt[i]== 0)[0]
    
        log_p_spam_i=np.log(p_spam)+sum(np.log(p_on_spam_m[spam_index]))+sum(np.log(1-p_on_spam_m[ham_index]))
        log_p_spam.append(log_p_spam_i)
    
        log_p_ham_i=np.log(p_ham)+sum(np.log(p_on_ham_m[spam_index]))+sum(np.log(1-p_on_ham_m[ham_index]))
        log_p_ham.append(log_p_ham_i)
    
    return np.array(log_p_spam), np.array(log_p_ham)

def accuarcy(y_true, y_pred):
    # calculate accuracy
    y_pred=y_pred*1
    result=sum(y_pred==y_true)/len(y_true)
    return result

## Load and preprocess data

In [86]:
from sklearn.feature_extraction import DictVectorizer
# load data
train_data, test_data = data_loader()

# extract labels to 0,1 and features to dicticnary
x_train, y_train, x_test, y_test = data_preprocessing(train_data, test_data)

# transform word dicts to feature vectors
vectorizer = DictVectorizer(sparse=False)

x_train_frt = vectorizer.fit_transform(x_train)
x_test_frt = vectorizer.transform(x_test)
#feature vector, columns are features, rows are training/testing data, entry value = number of occurences ith feature in jth record
#DictVectorizer will one hot encode categorical value and keep numerical value unchanged

print(x_train_frt.shape, x_test_frt.shape)

(9000, 1000) (1000, 1000)


### Grid search for the best $m$. 



In [93]:
def pipeline(x_train_frt, y_train, x_test_frt, y_test, m):
    p_spam, p_ham = compute_empirical_pmf_y(y_train)
    p_on_spam_m, p_on_ham_m = m_estimation_conditional_probability(x_train_frt, y_train, x_train_frt.shape[1], m)
    log_p_spam, log_p_ham = log_estimated_probability(p_spam, p_ham, p_on_spam_m, p_on_ham_m, x_test_frt)
    test_pred = (log_p_spam > log_p_ham)
    print(str(m) + ":" + str(accuarcy(y_test, test_pred)))

m_grid = [0,0.01, 0.1, 1, 10, 100, 1000]
for m in m_grid:
    pipeline(x_train_frt, y_train, x_test_frt, y_test, m)

<ipython-input-90-d43ed5e80bb7>:40: RuntimeWarning: divide by zero encountered in log
  log_p_spam_i=np.log(p_spam)+sum(np.log(p_on_spam_m[spam_index]))+sum(np.log(1-p_on_spam_m[ham_index]))


0:0.932
0.01:0.927
0.1:0.924
1:0.922
10:0.905
100:0.88
1000:0.845


In [94]:
#vs CountVectorizer
# >>> from sklearn.feature_extraction.text import CountVectorizer
# >>> corpus = [
# ...     'This is the first document.',
# ...     'This document is the second document.',
# ...     'And this is the third one.',
# ...     'Is this the first document?',
# ... ]
# >>> vectorizer = CountVectorizer()
# >>> X = vectorizer.fit_transform(corpus)
# >>> vectorizer.get_feature_names_out()
# array(['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third',
#        'this'], ...)
# >>> print(X.toarray())
# [[0 1 1 1 0 0 1 0 1]
#  [0 2 0 1 0 1 1 0 1]
#  [1 0 0 1 1 0 1 1 1]
#  [0 1 1 1 0 0 1 0 1]]
# >>> vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
# >>> X2 = vectorizer2.fit_transform(corpus)
# >>> vectorizer2.get_feature_names_out()
# array(['and this', 'document is', 'first document', 'is the', 'is this',
#        'second document', 'the first', 'the second', 'the third', 'third one',
#        'this document', 'this is', 'this the'], ...)
#  >>> print(X2.toarray())
#  [[0 0 1 1 0 0 1 0 0 0 0 1 0]
#  [0 1 0 1 0 1 0 1 0 0 1 0 0]
#  [1 0 0 1 0 0 0 0 1 1 0 1 0]
#  [0 0 1 0 1 0 1 0 0 0 0 0 1]]